In [ ]:
!pip install numpy pandas faiss-gpu torch transformers sentence_transformers --quiet

     |████████████████████████████████| 89.7MB 56kB/s 
     |████████████████████████████████| 2.5MB 45.5MB/s 
     |████████████████████████████████| 92kB 12.9MB/s 
     |████████████████████████████████| 901kB 39.8MB/s 
     |████████████████████████████████| 3.3MB 42.0MB/s 
     |████████████████████████████████| 1.2MB 39.7MB/s 


In [ ]:
data_folder = '/content/drive/MyDrive/ML-DL-DS/data/'

In [ ]:
import re
import numpy as np
import pandas as pd
import faiss
import torch
from torch.utils.data import DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration
from sentence_transformers import SentenceTransformer

np.random.seed(0)
pd.set_option("max_colwidth", 100)

In [ ]:
# wikipedia movie plots dataset: https://www.kaggle.com/jrobischon/wikipedia-movie-plots
movies = pd.read_csv(data_folder + 'wiki_movie_plots_deduped.csv', usecols=['Title', 'Plot'])

print(f"Plots of {len(movies.index)} movies!")
movies.sample(2)

Plots of 34886 movies!


,Title,Plot
23500,Belly of the Beast,Jake Hopper (Steven Seagal) is a successful businessman and retired CIA agent who runs a success...
26709,Payback,Insurance employee Kunal (Munish Khan) suffers an almost fatal accident but is rescued by passer...


In [ ]:
def clean_text(text, max_words=1024):
    """
    Truncates a string, then removes string control characters and multiple spaces.
    """
    text = text.split()[:max_words]
    text = ' '.join(text)
    regex = re.compile(r'[\n\r\t]') 
    text = regex.sub(" ", text)
    text = re.sub(' +', ' ', text).strip()
    return text

In [ ]:
clean_text("Lorem    ipsum dolor sit amet, consectetur adipiscing elit.\r\nSem integer vitae justo eget magna fermentum iaculis.")

'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sem integer vitae justo eget magna fermentum iaculis.'

In [ ]:
movies['Plot'] = movies['Plot'].apply(clean_text)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'  # use GPU if available
print(device)

cuda


In [ ]:
# summarization model
model = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-6')
model.to(device)
model.eval()

tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')

In [ ]:
FIRST_HP_MOVIE_PLOT = """Albus Dumbledore, Minerva McGonagall, and Rubeus Hagrid, professors of Hogwarts School of Witchcraft and Wizardry, deliver an orphaned infant named Harry Potter to his only remaining relatives, the Dursleys. Ten years later, Harry has been battling a disjointed life with the Dursleys, inadvertently causing an accident during a family outing, and begins receiving unsolicited letters by owls. Finally, Hagrid re-appears, and informs Harry that he is actually a wizard, and has been accepted into Hogwarts, against the Dursleys' wishes. He also tells Harry of the latter's past; Harry is the orphaned son of two wizards who met their demise at the hands of Lord Voldemort, a malevolent, all-powerful wizard, by a Killing Curse, with Harry being the only survivor in the chaos thus, leading to his fame in the wizarding world as "The Boy Who Lived". Hagrid takes Harry to Diagon Alley to purchase school supplies, then takes him to King's Cross station to board a train to the school. While on the train, Harry meets Ron Wea"""

In [ ]:
inputs = tokenizer([FIRST_HP_MOVIE_PLOT],
                   max_length=1024,
                   padding=True,
                   truncation=True,
                   return_tensors='pt')

# Generate Summary (max 128 tokens)
summary_ids = model.generate(inputs['input_ids'].to(device),
                             max_length=128,
                             early_stopping=True)

summaries = tokenizer.batch_decode(summary_ids,
                                   skip_special_tokens=True,
                                   clean_up_tokenization_spaces=True)

In [ ]:
summaries[0]

' Albus Dumbledore, Minerva McGonagall, and Rubeus Hagrid deliver Harry Potter to his only remaining relatives, the Dursleys. Harry is the orphaned son of two wizards who met their demise at the hands of Lord Voldemort, a malevolent, all-powerful wizard, by a Killing Curse. Harry was the only survivor in the chaos leading to his fame in the wizarding world as "The Boy Who Lived"'

In [ ]:
plot_list = movies.Plot.tolist()

def split_list(lst, n):  
    for i in range(0, len(lst), n): 
        yield lst[i:i + n] 

n = 32  # batch size
batches = list(split_list(plot_list, n))

In [ ]:
all_sumaries = []
for batch in batches:
    with torch.no_grad():
        # tokenize
        inputs = tokenizer(batch,
                        max_length=1024,
                        padding=True,
                        truncation=True, 
                        return_tensors='pt')

        # generate summary (max 128 tokens)
        summary_ids = model.generate(inputs['input_ids'].to(device), max_length=128, early_stopping=True).to('cpu')
        all_sumaries += [txt.strip() for txt in tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)]
        del inputs, summary_ids 
        torch.cuda.empty_cache()

movies['PlotSummary'] = all_sumaries

In [ ]:
movies.to_csv(data_folder + 'wiki_movie_plots_deduped_summarized.csv', index=False)

In [ ]:
movies.head()

,Title,Plot,PlotSummary
0,Kansas Saloon Smashers,"A bartender is working at a saloon, serving drinks to customers. After he fills a stereotypicall...",Carrie Nation and her followers burst into a saloon and attack a bartender. The group then begin...
1,Love by the Light of the Moon,"The moon, painted with a smiling face hangs over a park at night. A young couple walking past a ...","The moon, painted with a smiling face hangs over a park at night. A young couple walking past a ..."
2,The Martyred Presidents,"The film, just over a minute long, is composed of two shots. In the first, a girl sits at the ba...","The film, just over a minute long, is composed of two shots. In the first, a girl sits at the ba..."
3,"Terrible Teddy, the Grizzly King","Lasting just 61 seconds and consisting of two shots, the first shot is set in a wood during wint...","The first shot is set in a wood during winter and lasts just 61 seconds. Two men, bearing signs ..."
4,Jack and the Beanstalk,"The earliest known adaptation of the classic fairytale, this films shows Jack trading his cow fo...",The earliest known adaptation of the classic fairytale shows Jack trading his cow for the beans....


In [ ]:
pd.set_option("max_colwidth", None)
movies[movies.Title.str.contains('Harry Potter and the Order of the Phoenix')]

Title  ...                                                                                                                                                                                                                                                                                                                                                  PlotSummary
15225  Harry Potter and the Order of the Phoenix  ...  Harry is expelled from Hogwarts after using magic to save Dudley and himself from Dementors. The Ministry of Magic is oblivious to Lord Voldemort's return. Harry has a vision involving an attack upon Arthur Weasley, from the point of view of the attacker. The connection between Harry and Voldemort leads Harry to isolate himself from his friends.

[1 rows x 3 columns]